In [1]:
# Import required packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

pd.options.mode.chained_assignment = None  # default='warn'
import random

import meetup.api
import requests
import json
import time

Note: Group member count data is updated regularly and therefore will not tie to the member data unless updated together. There are also some members who are private and therefore will not have their information included which explains why certain totals (particularly in large meetups) will still not match.

In [2]:
# Add Meetup key and access API client
api_key = 'YOUR_API_KEY'
client = meetup.api.Client(api_key)
type(client)

meetup.api.Client

In [3]:
# Create function which creates list of London Technology meetups
# Meetup group criteria embedded in the API call
# Functionality is also built in to sleep in order to respect rate limits as Meetup API has variable rate limits
def create_group_df(n_groups):
    group_df = pd.DataFrame([])
    i = 0

    while i < n_groups:
        json_groups = requests.get(
            'https://api.meetup.com/find/groups?&sign=true&photo-host=public&country=GB&location=London&category=34&key={}&offset={}'.format(api_key,str(i)))
        group_df_dict = json_groups.json()
        temp_group_df = pd.DataFrame.from_dict(group_df_dict)
        group_df = pd.concat([group_df,temp_group_df],sort=False)
        i += 1

        print('----------Request {}----------'.format(i))
        print('Status: ',json_groups)
        print('X-RateLimit-Remaining: ',json_groups.headers['X-RateLimit-Remaining'])

        if int(json_groups.headers['X-RateLimit-Remaining']) == 5:
            time.sleep(int(json_groups.headers['X-RateLimit-Reset'])+5)

    group_df.reset_index(drop=True,inplace=True)
    return group_df

In [4]:
# Create desired number of groups
group_df = create_group_df(50)

# Remove groups with less than 50 members
group_df = group_df[group_df['members']>50]
group_df.reset_index(drop=True,inplace=True)

In [5]:
# Export group csv
group_df.to_csv('group_list.csv', index=False)

In [6]:
# Load prior member and group data (if revisiting this later)
#group_df = pd.read_csv('group_list.csv')
#member_df = pd.read_csv('member_list.csv')

In [11]:
# Select subset of groups of interest and only columns of interest which will be used to get members
sub_group_df = group_df[['name','urlname','id','members']].iloc[1301:1531]

# Alternatively can select groups which contain certain string in name (e.g. Python)
#sub_group_df = group_df[['name','urlname','id','members']].loc[group_df['name'].str.contains('Py')]
#group_df[['name','urlname','id','members']].iloc[:15]
print("Total Members: ",sub_group_df['members'].sum())
sub_group_df

Total Members:  175617


,name,urlname,id,members
1301,Vim London,Vim-London,5431952,503
1302,London NoSQL,London-NoSQL-and-Big-Data,12962072,1126
1303,HackHumanity,HackHumanity,12347062,233
1304,London Riak Meetup,riak-london,3738932,402
1305,Learn How to Make an App (iOS & Android),makeanapp,16147842,2312
1306,Big WP Meetup London,Big-Media-Enterprise-WordPress-London-Meetup,12502532,355
1307,InsTech London - The Insurance Technology Meetup,instech,18527736,2263
1308,Big Data Developers in London,Big-Data-Developers-in-London,9516272,7189
1309,RegTech - Capital Markets,RegTech,21830689,452
1310,Unofficial MuleSoft London Group,MuleSoft-Meetup-London,16649782,55


In [12]:
# Create function which queries API for members of a certain group by batch of up to 200
# Functionality is also built in to sleep in order to respect rate limits as Meetup API has variable rate limits
def create_member_df(group_id):
    member_df = pd.DataFrame([])
    i = 0
    number_of_requests = np.ceil(group_df['members'].loc[group_df['id'] == group_id] / 200).values
    
    while i < number_of_requests:
        try:
            json_members = requests.get('https://api.meetup.com/2/members?group_id={}&key={}&offset={}'.format(str(group_id), api_key,str(i)))
            member_df_dict = json_members.json()
            temp_member_df = pd.DataFrame.from_dict(member_df_dict['results'])
            member_df = pd.concat([member_df,temp_member_df],sort=False)
            i += 1

            print('----------Request {}----------'.format(i))
            print('Status: ',json_members)
            print('X-RateLimit-Remaining: ',json_members.headers['X-RateLimit-Remaining'])

            if int(json_members.headers['X-RateLimit-Remaining']) == 5:
                time.sleep(int(json_members.headers['X-RateLimit-Reset'])+5)
            member_df.reset_index(drop=True,inplace=True)
        except:
            i += 1

    return member_df

In [13]:
member_df = pd.DataFrame([])

# Run member function for all groups in subgroup and create dataframe out of members
for group_id in sub_group_df['id']:
    try:
        temp_member_df = create_member_df(group_id)
        temp_member_df['group_name'] = sub_group_df['urlname'].loc[sub_group_df['id'] == group_id].item()
        member_df = pd.concat([member_df,temp_member_df],sort=False)
    except:
        continue

----------Request 1----------
Status:  <Response [200]>
X-RateLimit-Remaining:  29
----------Request 2----------
Status:  <Response [200]>
X-RateLimit-Remaining:  28
----------Request 3----------
Status:  <Response [200]>
X-RateLimit-Remaining:  27
----------Request 1----------
Status:  <Response [200]>
X-RateLimit-Remaining:  26
----------Request 2----------
Status:  <Response [200]>
X-RateLimit-Remaining:  25
----------Request 3----------
Status:  <Response [200]>
X-RateLimit-Remaining:  29
----------Request 4----------
Status:  <Response [200]>
X-RateLimit-Remaining:  28
----------Request 5----------
Status:  <Response [200]>
X-RateLimit-Remaining:  27
----------Request 6----------
Status:  <Response [200]>
X-RateLimit-Remaining:  26
----------Request 1----------
Status:  <Response [200]>
X-RateLimit-Remaining:  25
----------Request 2----------
Status:  <Response [200]>
X-RateLimit-Remaining:  29
----------Request 1----------
Status:  <Response [200]>
X-RateLimit-Remaining:  28
----

----------Request 1----------
Status:  <Response [200]>
X-RateLimit-Remaining:  26
----------Request 2----------
Status:  <Response [200]>
X-RateLimit-Remaining:  25
----------Request 1----------
Status:  <Response [200]>
X-RateLimit-Remaining:  24
----------Request 2----------
Status:  <Response [200]>
X-RateLimit-Remaining:  29
----------Request 1----------
Status:  <Response [200]>
X-RateLimit-Remaining:  28
----------Request 2----------
Status:  <Response [200]>
X-RateLimit-Remaining:  27
----------Request 3----------
Status:  <Response [200]>
X-RateLimit-Remaining:  26
----------Request 4----------
Status:  <Response [200]>
X-RateLimit-Remaining:  25
----------Request 5----------
Status:  <Response [200]>
X-RateLimit-Remaining:  24
----------Request 6----------
Status:  <Response [200]>
X-RateLimit-Remaining:  29
----------Request 7----------
Status:  <Response [200]>
X-RateLimit-Remaining:  28
----------Request 1----------
Status:  <Response [200]>
X-RateLimit-Remaining:  27
----

----------Request 1----------
Status:  <Response [200]>
X-RateLimit-Remaining:  29
----------Request 1----------
Status:  <Response [200]>
X-RateLimit-Remaining:  28
----------Request 2----------
Status:  <Response [200]>
X-RateLimit-Remaining:  27
----------Request 3----------
Status:  <Response [200]>
X-RateLimit-Remaining:  26
----------Request 1----------
Status:  <Response [200]>
X-RateLimit-Remaining:  25
----------Request 2----------
Status:  <Response [200]>
X-RateLimit-Remaining:  29
----------Request 1----------
Status:  <Response [200]>
X-RateLimit-Remaining:  28
----------Request 1----------
Status:  <Response [200]>
X-RateLimit-Remaining:  27
----------Request 2----------
Status:  <Response [200]>
X-RateLimit-Remaining:  26
----------Request 3----------
Status:  <Response [200]>
X-RateLimit-Remaining:  25
----------Request 4----------
Status:  <Response [200]>
X-RateLimit-Remaining:  24
----------Request 5----------
Status:  <Response [200]>
X-RateLimit-Remaining:  29
----

----------Request 8----------
Status:  <Response [200]>
X-RateLimit-Remaining:  26
----------Request 9----------
Status:  <Response [200]>
X-RateLimit-Remaining:  25
----------Request 10----------
Status:  <Response [200]>
X-RateLimit-Remaining:  24
----------Request 11----------
Status:  <Response [200]>
X-RateLimit-Remaining:  29
----------Request 12----------
Status:  <Response [200]>
X-RateLimit-Remaining:  28
----------Request 1----------
Status:  <Response [200]>
X-RateLimit-Remaining:  27
----------Request 2----------
Status:  <Response [200]>
X-RateLimit-Remaining:  26
----------Request 3----------
Status:  <Response [200]>
X-RateLimit-Remaining:  25
----------Request 4----------
Status:  <Response [200]>
X-RateLimit-Remaining:  29
----------Request 1----------
Status:  <Response [200]>
X-RateLimit-Remaining:  28
----------Request 1----------
Status:  <Response [200]>
X-RateLimit-Remaining:  27
----------Request 2----------
Status:  <Response [200]>
X-RateLimit-Remaining:  26
-

----------Request 2----------
Status:  <Response [200]>
X-RateLimit-Remaining:  29
----------Request 3----------
Status:  <Response [200]>
X-RateLimit-Remaining:  28
----------Request 1----------
Status:  <Response [200]>
X-RateLimit-Remaining:  27
----------Request 2----------
Status:  <Response [200]>
X-RateLimit-Remaining:  26
----------Request 1----------
Status:  <Response [200]>
X-RateLimit-Remaining:  25
----------Request 2----------
Status:  <Response [200]>
X-RateLimit-Remaining:  24
----------Request 3----------
Status:  <Response [200]>
X-RateLimit-Remaining:  23
----------Request 4----------
Status:  <Response [200]>
X-RateLimit-Remaining:  22
----------Request 5----------
Status:  <Response [200]>
X-RateLimit-Remaining:  29
----------Request 6----------
Status:  <Response [200]>
X-RateLimit-Remaining:  28
----------Request 7----------
Status:  <Response [200]>
X-RateLimit-Remaining:  27
----------Request 8----------
Status:  <Response [200]>
X-RateLimit-Remaining:  26
----

----------Request 2----------
Status:  <Response [200]>
X-RateLimit-Remaining:  29
----------Request 3----------
Status:  <Response [200]>
X-RateLimit-Remaining:  28
----------Request 4----------
Status:  <Response [200]>
X-RateLimit-Remaining:  27
----------Request 1----------
Status:  <Response [200]>
X-RateLimit-Remaining:  26
----------Request 2----------
Status:  <Response [200]>
X-RateLimit-Remaining:  25
----------Request 3----------
Status:  <Response [200]>
X-RateLimit-Remaining:  24
----------Request 4----------
Status:  <Response [200]>
X-RateLimit-Remaining:  23
----------Request 5----------
Status:  <Response [200]>
X-RateLimit-Remaining:  29
----------Request 6----------
Status:  <Response [200]>
X-RateLimit-Remaining:  28
----------Request 1----------
Status:  <Response [200]>
X-RateLimit-Remaining:  27
----------Request 1----------
Status:  <Response [200]>
X-RateLimit-Remaining:  26
----------Request 2----------
Status:  <Response [200]>
X-RateLimit-Remaining:  25
----

----------Request 1----------
Status:  <Response [200]>
X-RateLimit-Remaining:  26
----------Request 1----------
Status:  <Response [200]>
X-RateLimit-Remaining:  25
----------Request 2----------
Status:  <Response [200]>
X-RateLimit-Remaining:  24
----------Request 1----------
Status:  <Response [200]>
X-RateLimit-Remaining:  23
----------Request 2----------
Status:  <Response [200]>
X-RateLimit-Remaining:  29
----------Request 3----------
Status:  <Response [200]>
X-RateLimit-Remaining:  28
----------Request 1----------
Status:  <Response [200]>
X-RateLimit-Remaining:  27
----------Request 2----------
Status:  <Response [200]>
X-RateLimit-Remaining:  26
----------Request 1----------
Status:  <Response [200]>
X-RateLimit-Remaining:  25
----------Request 1----------
Status:  <Response [200]>
X-RateLimit-Remaining:  24
----------Request 2----------
Status:  <Response [200]>
X-RateLimit-Remaining:  23
----------Request 1----------
Status:  <Response [200]>
X-RateLimit-Remaining:  22
----

----------Request 8----------
Status:  <Response [200]>
X-RateLimit-Remaining:  25
----------Request 9----------
Status:  <Response [200]>
X-RateLimit-Remaining:  24
----------Request 10----------
Status:  <Response [200]>
X-RateLimit-Remaining:  23
----------Request 11----------
Status:  <Response [200]>
X-RateLimit-Remaining:  29
----------Request 1----------
Status:  <Response [200]>
X-RateLimit-Remaining:  28
----------Request 2----------
Status:  <Response [200]>
X-RateLimit-Remaining:  27
----------Request 1----------
Status:  <Response [200]>
X-RateLimit-Remaining:  26
----------Request 2----------
Status:  <Response [200]>
X-RateLimit-Remaining:  25
----------Request 3----------
Status:  <Response [200]>
X-RateLimit-Remaining:  24
----------Request 1----------
Status:  <Response [200]>
X-RateLimit-Remaining:  23
----------Request 1----------
Status:  <Response [200]>
X-RateLimit-Remaining:  22
----------Request 2----------
Status:  <Response [200]>
X-RateLimit-Remaining:  29
--

----------Request 3----------
Status:  <Response [200]>
X-RateLimit-Remaining:  28
----------Request 1----------
Status:  <Response [200]>
X-RateLimit-Remaining:  27
----------Request 2----------
Status:  <Response [200]>
X-RateLimit-Remaining:  26
----------Request 1----------
Status:  <Response [200]>
X-RateLimit-Remaining:  25
----------Request 1----------
Status:  <Response [200]>
X-RateLimit-Remaining:  24
----------Request 1----------
Status:  <Response [200]>
X-RateLimit-Remaining:  23
----------Request 1----------
Status:  <Response [200]>
X-RateLimit-Remaining:  29
----------Request 2----------
Status:  <Response [200]>
X-RateLimit-Remaining:  28
----------Request 1----------
Status:  <Response [200]>
X-RateLimit-Remaining:  27
----------Request 2----------
Status:  <Response [200]>
X-RateLimit-Remaining:  26
----------Request 1----------
Status:  <Response [200]>
X-RateLimit-Remaining:  25
----------Request 2----------
Status:  <Response [200]>
X-RateLimit-Remaining:  24
----

----------Request 2----------
Status:  <Response [200]>
X-RateLimit-Remaining:  26
----------Request 3----------
Status:  <Response [200]>
X-RateLimit-Remaining:  25
----------Request 4----------
Status:  <Response [200]>
X-RateLimit-Remaining:  29
----------Request 5----------
Status:  <Response [200]>
X-RateLimit-Remaining:  28
----------Request 6----------
Status:  <Response [200]>
X-RateLimit-Remaining:  27
----------Request 7----------
Status:  <Response [200]>
X-RateLimit-Remaining:  26
----------Request 8----------
Status:  <Response [200]>
X-RateLimit-Remaining:  25
----------Request 9----------
Status:  <Response [200]>
X-RateLimit-Remaining:  29
----------Request 10----------
Status:  <Response [200]>
X-RateLimit-Remaining:  28
----------Request 11----------
Status:  <Response [200]>
X-RateLimit-Remaining:  27
----------Request 12----------
Status:  <Response [200]>
X-RateLimit-Remaining:  26
----------Request 13----------
Status:  <Response [200]>
X-RateLimit-Remaining:  25


In [14]:
# Spot check total number of members in subset of groups and number of members in certain groups 
print("Total Members :",len(member_df))
member_df['group_name'].value_counts()

Total Members : 170591


Big-Data-Developers-in-London                             7193
DevOps-Exchange-London                                    6590
London-Startup-Founder-101                                6363
londonweb                                                 4687
ln-enterprise-tech                                        4264
agiletesting                                              4083
geekpub                                                   3286
LondonR                                                   3228
London-Startup-Growth-Meetup                              3137
Creative-Class                                            3006
Jenkins-online-meetup                                     2756
Python-for-Quant-Finance-London                           2675
AppliedAI                                                 2558
Data-Natives-London                                       2553
UX-Playground-The-User-Experience-Meetup-for-Londoners    2372
makeanapp                                              

In [15]:
# Create member list dataframe
member_df.to_csv('1301-1531_member_list.csv', index=False)

In [17]:
# Retrieve missing group names (some groups are private) and ensure it satisfactory
set1 = set(list(group_df['urlname'].iloc[1301:1531].sort_values(ascending=False)))
set2 = set(list(member_df['group_name'].value_counts().index))
set1.symmetric_difference(set2)

{'BudEvents',
 'Cpp-London-Developers',
 'Digital-Adventure-Club',
 'Enterprise-UX-London',
 'Financial-Engineers-Quants-London',
 'London-CRM-Meetup',
 'London-Engineers-Artists-Meetup',
 'RegTech',
 'london-seo-ppc',
 'werinnovation'}